# Environment

In [1]:
import $ivy.`org.scala-lang.modules:scala-parser-combinators_2.12:2.3.0`

import scala.util.parsing.combinator._

import $ivy.$                                                           


import scala.util.parsing.combinator._

# Test Datas

In [5]:
val simpleTestData = """--algorithm EuclidAlg
variables u = 24 , v \in 1 .. N , v_ini = v
begin while u # 0 do
        if u < v then u := v || v := u; \* swap u and v
            end if;
        u := u - v;
      end while ;
      print <<24, v_ini, "have gcd", v>> ;
end algorithm"""

simpleTestData: String = """--algorithm EuclidAlg
variables u = 24 , v \in 1 .. N , v_ini = v
begin while u # 0 do
        if u < v then u := v || v := u; \* swap u and v
            end if;
        u := u - v;
      end while ;
      print <<24, v_ini, "have gcd", v>> ;
end algorithm"""

In [4]:
val complextTestData = """\* Figure 2: The fast mutual exclusion algorithm in PlusCal.
--algorithm FastMutex
variables x , y = 0 , b = [i \in 1..N |-> FALSE]

process Proc \in 1..N
    variable j ;
    begin
        ncs: while TRUE do
                 skip ; \∗ The noncritical section.
          start: b[self] := TRUE ;
             l1: x := self ;
             l2: if y # 0 then l3: b[self] := FALSE ;
                                 l4: await y = 0 ;
                                     goto start
                end if ;
             l5: y := self ;
             l6: if x # self
                   then l7: b[self] := FALSE ;
                            j := 1 ;
                     l8: while j ≤ N do await ~b[j] ;
                                        j := j+1
                         end while ;
                     l9: if y 6 = self then l10: await y = 0 ;
                                                 goto start ;
                         end if ;
                   end if;
             cs: skip ; \∗ The critical section.
            l11: y := 0 ;
            l12: b[self] := FALSE ;
        end while ;
end process
end algorithm
"""

complextTestData: String = """\* Figure 2: The fast mutual exclusion algorithm in PlusCal.
--algorithm FastMutex
variables x , y = 0 , b = [i \in 1..N |-> FALSE]

process Proc \in 1..N
    variable j ;
    begin
        ncs: while TRUE do
                 skip ; \∗ The noncritical section.
          start: b[self] := TRUE ;
             l1: x := self ;
             l2: if y # 0 then l3: b[self] := FALSE ;
                                 l4: await y = 0 ;
                                     goto start
                end if ;
             l5: y := self ;
             l6: if x # self
                   then l7: b[self] := FALSE ;
                            j := 1 ;
                     l8: while j ≤ N do await ~b[j] ;
                                        j := j+1
                         end while ;
                     l9: if y 6 = self then l10: await y = 0 ;
                                                 goto start ;
                         end if ;
                   end if;

# A Builder?

In [6]:
class PlusCalProgram extends JavaTokenParsers {
    def algoritm: Parser[Any] = ("--algorithm" | "--fair" ~ "algorithm") ~ 
    name ~ opt(varDecls) ~ 
    opt(definitions) ~ 
    rep(_macro) ~ 
    rep(procedure) ~ 
    opt((algorithmBody | rep1(process))) ~ 
    "end algorithm"
    
    def definitions: Parser[Any] = "define" ~ defs ~ "end" ~ "define" ~ opt(";")
    def _macro: Parser[Any] = "macro" ~ name ~ opt(repsep(variable, ",")) ~ algorithmBody ~ "end" ~ "macro" ~ opt(";")
    def procedure: Parser[Any] = "procedure" ~ name ~ opt(repsep(pVarDecl, ",")) ~ opt(pVarDecls) ~ algorithmBody ~ "end" ~ "procedure" ~ opt(";")
    def process: Parser[Any] = opt("fair" ~ opt("+")) ~ "process" ~ name ~ ("=" | "\\in") ~ expr ~ opt(varDecls) ~ algorithmBody ~ "end" ~ "process" ~ opt(";")
    def varDecls: Parser[Any] = ("varaible" | "variables") ~ varDecl
    def varDecl: Parser[Any] = variable ~ opt(("=" | "\\in") ~ expr) ~ (";" | ",")
    def pVarDecls: Parser[Any] = ("varaible" | "variables") ~ rep1(pVarDecl ~ (";" | ","))
    def pVarDecl: Parser[Any] = variable ~ opt("=" ~ expr)
    def algorithmBody: Parser[Any] = "begin" ~ opt(stmt)
    def stmt: Parser[Any] = opt(label ~ ":" ~ opt("+" | "-")) ~ unlabeledStmt
    def unlabeledStmt: Parser[Any] = assign | _if | _while | either | _with | await | _print | _assert | skip | _return | goto | call | macroCall
    def assign: Parser[Any] = LHS ~ ":=" ~ expr ~ rep("||" ~ LHS ~ ":=" ~ expr)
    def LHS: Parser[Any] = variable ~ rep(repsep(expr, ",") | "." ~ field)
    def _if: Parser[Any] = "if" ~ expr ~ "then" ~ rep1(stmt) ~ rep("elsif" ~ expr ~ "then" ~ rep1(stmt)) ~ opt("else" ~ rep1(stmt)) ~ "end" ~ "if" ~ ";"
    def _while: Parser[Any] = "while" ~ expr ~ "do" ~ rep1(stmt) ~ "end" ~ "while"
    def either: Parser[Any] = "either" ~ rep1(stmt) ~ rep1("or" ~ rep1(stmt)) ~ "end" ~ "either" ~ ";"
    def _with: Parser[Any] = "with" ~ rep1(variable ~ ("=" | "\\in") ~ expr ~ (";" | ",")) ~ "do" ~ rep1(stmt) ~ "end" ~ "with" ~ ";"
    def await: Parser[Any] = ("await" | "when") ~ expr
    def _print: Parser[Any] = "print" ~ expr
    def _assert: Parser[Any] = "assert" ~ expr
    def skip: Parser[Any] = "skip" ~ ";"
    def _return: Parser[Any] = "return" ~ ";"
    def goto: Parser[Any] = "goto" ~ label
    def call: Parser[Any] = "call" ~ macroCall
    def macroCall: Parser[Any] = name ~ opt(rep1(expr)) ~ ";"
    
    def variable: Parser[Any] = stringLiteral
    def field: Parser[Any] = stringLiteral
    def name: Parser[String] = stringLiteral
    def label: Parser[String] = stringLiteral
    def expr: Parser[Any] = stringLiteral
    def defs: Parser[Any] = stringLiteral
}

defined class PlusCalProgram

In [7]:
val program = new PlusCalProgram
program.parseAll(program.algoritm, simpleTestData)

program: PlusCalProgram = ammonite.$sess.cmd5$Helper$PlusCalProgram@f165823
res6_1: program.ParseResult[Any] = Failure(
  "string matching regex '\"([^\"\\x00-\\x1F\\x7F\\\\]|\\\\[\\\\'\"bfnrt]|\\\\u[a-fA-F0-9]{4})*\"' expected but 'E' found",
  CharSequenceReader('E', ...)
)